## Libraries

In [1]:
#from accelerate import PartialState
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
import torch

/media/mujtaba/DATA/nick/miniconda3/envs/UnitTestGeneration/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Creating LORA and Quantization Configs

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

## DeepSeek-Coder-V2-Lite-Instruct

In [3]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct")

model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct:
- configuration_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct:
- modeling_deepseek.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [03:06<00:00, 46.57s/it]


## CodeLlama-13b-Instruct-hf

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-13b-Instruct-hf")
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-13b-Instruct-hf",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True, #Automatic with quantized models
)

del(model)
del(tokenizer)

Loading checkpoint shards: 100%|██████████| 3/3 [01:05<00:00, 21.68s/it]


## SantaCoder

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigcode/santacoder", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "bigcode/santacoder",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)
del(model)
del(tokenizer)

A new version of the following files was downloaded from https://huggingface.co/bigcode/santacoder:
- configuration_gpt2_mq.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/bigcode/santacoder:
- modeling_gpt2_mq.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


## Instruct CodeT5P

In [7]:
from transformers import AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/instructcodet5p-16b")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "Salesforce/instructcodet5p-16b",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

A new version of the following files was downloaded from https://huggingface.co/Salesforce/instructcodet5p-16b:
- modeling_codet5p.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
CodeT5pForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an e

## CodeGen2.5-7b-Instruct_P

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen25-7b-instruct_P")
model = AutoModelForCausalLM.from_pretrained(
    "Salesforce/codegen25-7b-instruct_P",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

AttributeError: add_special_tokens conflicts with the method add_special_tokens in CodeGen25Tokenizer

## WizardCoder-15B-V1.0

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("WizardLMTeam/WizardCoder-15B-V1.0")
model = AutoModelForCausalLM.from_pretrained(
    "WizardLMTeam/WizardCoder-15B-V1.0",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

## StarCoder2-15b-instruct-v0.1

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("WizardLMTeam/WizardCoder-15B-V1.0")
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-13b-Instruct-hf",
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

del(model)
del(tokenizer)

## Freeing Up Cache Space

In [ ]:
#Delete models from cache
import subprocess

subprocess.run(["rm", "-rf", "~/.cache/huggingface/hub"])

In [ ]:
#Delete datasets from cache (Shouldn't be needed with this data)
import subprocess

subprocess.run(["rm", "-rf", "~/.cache/huggingface/datasets"])